In [ ]:
import tensorflow

In [ ]:
%load_ext autoreload
%autoreload 2
#%tensorflow_version 1.x
import tensorflow 
import numpy as np
 
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, LSTM, Dense
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import (Reshape,Activation,Attention,MaxPool1D,Dense, Conv1D, Convolution2D, GRU, LSTM, Lambda, Bidirectional, TimeDistributed,
                          Dropout, Flatten, LayerNormalization,RepeatVector, Reshape, MaxPooling1D, UpSampling1D, BatchNormalization)
import tensorflow.keras.layers as layers
import string
from tensorflow.keras.regularizers import l1, l2
import tensorflow.keras.backend as K
from tensorflow.keras.optimizers import Adam
import tensorflow as tf
import tensorflow.keras as keras
import datetime

import tensorflow as tf
import tensorflow_probability as tfp

import numpy as np
import sys
#sys.path.append('/content/pyNSID/')
#import pyNSID as nsid
import matplotlib.pyplot as plt
#import sidpy as sid
#import h5py
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.cluster import KMeans
from os.path import join as pjoin
from scipy import ndimage

In [ ]:
tensorflow.__version__

In [ ]:
import os
def make_folder(folder, **kwargs):
    """
    Function that makes new folders
    Parameters
    ----------'
    folder : string
        folder where to save
    Returns
    -------
    folder : string
        folder where to save
    """

    # Makes folder
    os.makedirs(folder, exist_ok=True)

    return (folder)

In [ ]:
pie_data = np.load('pizoresponse.npy')

In [ ]:
res_data = np.load('resonance.npy')

In [ ]:
vol = np.load('voltage.npy')

In [ ]:
res_data = np.atleast_3d(res_data)

In [ ]:
new_data = np.concatenate((pie_data,res_data),axis=2)

In [ ]:
class Sampling(layers.Layer):
  """Uses (z_mean, z_log_var) to sample z, the vector encoding a digit."""

  def call(self, inputs):
    z_mean, z_log_var = inputs
    batch = tf.shape(z_mean)[0]
    dim = tf.shape(z_mean)[1]
    epsilon = tf.keras.backend.random_normal(shape=(batch, dim))
    return z_mean + tf.exp(0.5 * z_log_var) * epsilon

class model_builder:
    
    def __init__(self,
                 input_data, 
                drop_frac = 0.2, 
                 layer_size = 128, 
                 num_ident_blocks = 3, 
                 l1_norm = 0,
                 l1_norm_embedding = 1e-3, 
                 layer_steps = 2,
                 embedding = 16, 
                 VAE = True,
                 coef = 1):
    # Sets self.mean and self.std to use in the loss function;
#       self.mean = 0
#       self.std = 0
    
      # Sets the L1 norm on the decoder/encoder layers
      self.l1_norm = l1_norm
 
      # Sets the fraction of dropout
      self.drop_frac = drop_frac
      
      # saves the shape of the input data
      self.data_shape = input_data.shape
 
      # Sets the number of neurons in the encoder/decoder layers
      self.layer_size = layer_size
 
      # Sets the number of neurons in the embedding layer
      self.embedding = embedding 
 
      # Bool to set if the model is a VAE 
      self.VAE = VAE
 
      # Set the magnitude of the l1 regularization on the embedding layer.
      self.l1_norm_embedding = l1_norm_embedding
 
      # sets the number of layers between the residual layer
      self.layer_steps = layer_steps

      self.coef = coef
 
      # set the number of identity block
      self.num_ident_blocks = num_ident_blocks
 
      self.model_constructor(input_data)
 
    def identity_block(self, X, name, 
                      block):
      
      # sets the name of the conv layers
      LSTM_name_base =  name + '_LSTM_Res_' + block
      bn_name_base = name + '_layer_norm_' + block
 
 
      # output for the residual layer
      X_shortcut = X
 
      for i in range(self.layer_steps):
            # bidirectional LSTM
            X = layers.Bidirectional(LSTM(self.layer_size, 
                                          return_sequences=True, 
                                          dropout=self.drop_frac,
                                          activity_regularizer=l1(self.l1_norm)), 
                                    input_shape=(self.data_shape[1]*2, 1))(X)

            # TODO, We could add layer norm
            X = layers.Activation('relu')(X)  

      X = layers.add([X, X_shortcut])
  #    X = layers.LayerNormalization(axis = 1, name = bn_name_base + '_res_end')(X)
      X = layers.Activation('relu')(X)
 
      return X
 
    def model_constructor(self, input_data):
      # defines the input
      encoder_input = layers.Input(shape=(self.data_shape[1:]))
      X = layers.Flatten()(encoder_input)
      X = layers.RepeatVector(1)(X)
      X = layers.Permute((2,1))(X)

#      X = encoder_input
      
      for i in range(self.num_ident_blocks):
        X = self.identity_block(X, 'encoder', string.ascii_uppercase[i+1])
        
      # This is in preparation for the embedding layer
      X = layers.Bidirectional(LSTM(self.layer_size, 
                                    return_sequences=False, 
                                    dropout=self.drop_frac,
                                    activity_regularizer=l1(self.l1_norm)), 
                                    input_shape=(self.data_shape[1]*2, 
                                                 1))(X)   
      
 #     X = layers.BatchNormalization(axis=1, name='last_encode')(X)
      X = layers.Activation('relu')(X)

      if self.VAE:
            X = layers.Dense(self.embedding, name="embedding_pre")(X)
            X = layers.Activation('relu')(X)
            X = layers.ActivityRegularization(l1=self.l1_norm_embedding*10**(self.coef))(X)
            z_mean = layers.Dense(self.embedding, name="z_mean")(X)
            z_log_var = layers.Dense(self.embedding, name="z_log_var")(X)
            self.sampling = Sampling()((z_mean, z_log_var))
            # update the self.mean and self.std:
#            self.mean = z_mean
#            self.std = z_log_var

      self.encoder_model = Model(inputs=encoder_input, outputs=self.sampling, name='LSTM_encoder')

      decoder_input = layers.Input(shape=(self.embedding,), name="z_sampling")

      z = layers.Dense(self.embedding, name="embedding")(decoder_input)
      z = layers.Activation('relu')(z)
      z = layers.ActivityRegularization(l1=self.l1_norm_embedding*10**(self.coef))(z)

      X = layers.RepeatVector(self.data_shape[1])(z)
 
      X = layers.Bidirectional(LSTM(self.layer_size, return_sequences=True, 
                                    dropout=self.drop_frac,
                                    activity_regularizer=l1(self.l1_norm)))(X)
 
      # X = layers.BatchNormalization(axis = 1, name = 'fires_decode')(X)
      X = layers.Activation('relu')(X)
 
      for i in range(self.num_ident_blocks):
        X = self.identity_block(X, 'decoder', string.ascii_uppercase[i+1])
    
 
 #     X = layers.LayerNormalization(axis=1, name='batch_normal')(X)
      X = layers.TimeDistributed(Dense(2, activation='linear'))(X)

      self.decoder_model = Model(inputs=decoder_input, outputs=X, name='LSTM_encoder')

      outputs = self.decoder_model(self.sampling)

      self.vae = tf.keras.Model(inputs=encoder_input, outputs=outputs, name="vae")

      # Add KL divergence regularization loss.
      kl_loss = -0.5 * tf.reduce_mean(z_log_var - tf.square(z_mean) - tf.exp(z_log_var) + 1)
      self.vae.add_loss(self.coef*kl_loss)

In [ ]:
epochs=62000 
epoch_per_increase = 1000
iteration = (epochs//epoch_per_increase) + 1
iteration

In [ ]:
def Train(epochs, epoch_per_increase, initial_beta, beta_per_increase):
    best_loss = float('inf')
    iteration = (epochs//epoch_per_increase) + 1
    model = []
    #filepath =folder + '/if_appear_means_bug_happens.hdf5'
    
    for i in range(iteration):
        if i == iteration-1:
            training_epochs = epochs - epoch_per_increase*(iteration-1)
            if training_epochs <= 0:
                break
        else:
            training_epochs = epoch_per_increase
            
        beta = initial_beta + beta_per_increase*i
        print(beta)
        del(model)
        model = model_builder(np.atleast_3d(new_data),embedding=16, 
                      VAE=True, l1_norm_embedding=1e-5,coef=beta)
        run_id = '_beta_step_siez=0.0025_' + np.str(model.embedding) + '_layer_size_' + np.str(model.layer_size) + '_l1_norm_' + np.str(model.l1_norm) + '_l1_norm_' + np.str(model.l1_norm_embedding) + '_VAE_' + np.str(model.VAE)
        folder = make_folder('piezoresponse+resonacnce/'+'beta='+np.str(beta)+ '_'+run_id)
#        if i==30:
#            filepath = 'piezoresponse+resonacnce_1/beta=0.0725__beta_step_siez=0.0025_16_layer_size_128_l1_norm_0_l1_norm_1e-05_VAE_True/triple_phase_weights2_epochs=29.hdf5'
        if i >0:
            print(filepath)
            model.vae.load_weights(filepath)
        else:
            training_epochs = 6000
          #model.vae.load_weights('/content/drive/My Drive/papers/Faster_Better_Paper/uniform/0.6000000000000001_V10_fast_100_epochs_per_step_dense_L1_phase_shift_Notebook_w_layernorm_VAE (modified_2)16_layer_size_128_l1_norm_0_l1_norm_0.0001_VAE_True/triple_phase_weights_beta_0.6000000000000001_epochs_begin_450+0048-0.08882.hdf5')
            
        optimizer = tf.keras.optimizers.Adam(learning_rate=3e-5)
        model.vae.compile(optimizer, loss=tf.keras.losses.MeanSquaredError())
        
#        beta = beta + i*beta_per_increase
        # sets the file path
        epoch_begin = i*epoch_per_increase
        if i>0:
            filepath = folder + '/phase_shift_only' + np.str(beta)+'_epochs_begin_6000+'+np.str(epoch_begin)+'+{epoch:04d}'+'-{loss:.5f}.hdf5'
        else:
            
            filepath = folder + '/phase_shift_only' + np.str(beta)+'_epochs_begin_'+np.str(epoch_begin)+'+{epoch:04d}'+'-{loss:.5f}.hdf5'

        # callback for saving checkpoints. Checkpoints are only saved when the model improves
        checkpoint = keras.callbacks.ModelCheckpoint(filepath, monitor='loss',
                                                      verbose=0, save_best_only=True,
                                                      save_weights_only=True, mode='min')

#         if i==0:
            
#             model.vae.compile(optimizer, loss=KL_Loss(0,0,beta))
#         else:
#             model.vae.compile(optimizer, loss=KL_Loss(model.mean,model.std,beta))
        model.vae.fit(np.atleast_3d(new_data),
                      np.atleast_3d(new_data),
                      batch_size=180, epochs=training_epochs,callbacks=[checkpoint])
    
#        total_loss = hist.history['loss'][0]
    
#        best_loss = total_loss
        filepath = folder + '/triple_phase_weights2_epochs='+np.str(i)+'.hdf5'
        model.vae.save_weights(filepath)

In [ ]:
from numpy.random import seed
seed(1)
Train(epochs,epoch_per_increase,0,0.0025)